#  로지스틱 회귀 앙상블

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

모델선언

In [2]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return out


하이퍼 파라미터 설정

In [3]:
input_size = 28 * 28
num_classes = 10
num_epochs = 10
learning_rate = 0.001
num_models = 5 # 앙상블에 사용할 모델의 개수

 데이터셋, 데이터 로더 생성 

In [4]:
dataset = FashionMNIST(root="./data/", train=True, transform=ToTensor(), download=False)
train_set, val_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=100, shuffle=True)
val_loader = DataLoader(val_set, batch_size=100, shuffle=False)

앙상블

In [5]:
# 앙상블 모델 생성 
models = [LogisticRegression(input_size, num_classes) for _ in range(num_models)]
print(models)

[LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
), LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)]


모델, 손실함수, 옵티마이저 정의 

In [6]:
criterion = nn.CrossEntropyLoss()
optimizers = [optim.SGD(model.parameters(), lr=learning_rate) for model in models ]

학습

In [7]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        # 데이터 로드
        images = images.reshape(-1, input_size)
        labels = labels
        
        # 순전파 및 손실 계산
        for j in range(num_models):
            outputs = models[j](images)
            loss = criterion(outputs, labels)
            
            # 역전파 및 가중치 업데이트 
            optimizers[j].zero_grad()
            loss.backward()
            optimizers[j].step()
            
            
    # 검증 
    with torch.no_grad():
        total, correct = 0, 0
        for images, labels in val_loader:
            images = images.reshape(-1,  input_size)
            outputs = torch.zeros(images.size()[0], num_classes)
            """
            outputs = torch.zeros(images.size()[0], num_classes)
            이미지  배치에 대한 출력 텐서 초기화 
            >> 후속 단계에서 이미지에 대한 예측값 업데이트 기능 
            
            """

            # 앙상블 모델의 예측값을 더함
            for j in range(num_models):
                outputs += models[j](images)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy:{val_acc:.2f}%")

Epoch [1/10], Validation Accuracy:65.33%
Epoch [2/10], Validation Accuracy:66.22%
Epoch [3/10], Validation Accuracy:67.20%
Epoch [4/10], Validation Accuracy:68.38%
Epoch [5/10], Validation Accuracy:69.23%
Epoch [6/10], Validation Accuracy:70.35%
Epoch [7/10], Validation Accuracy:71.26%
Epoch [8/10], Validation Accuracy:72.23%
Epoch [9/10], Validation Accuracy:72.95%
Epoch [10/10], Validation Accuracy:73.31%
